#### Kay Avila<br />IS 445, Week 8 - Maps<br />March 20, 2022
Resources used beyond the standard library documentation:
* https://matplotlib.org/3.5.1/tutorials/intermediate/arranging_axes.html#high-level-methods-for-making-grids

In [1]:
import os
import json
import bqplot
import requests
import geopandas
import ipywidgets
import pandas as pd
import contextily as ctx
import matplotlib.pyplot as plt

### Plot the trees owned by the city of Champaign

In [2]:
# Read the Champaign city council data into memory
gdf_councils = geopandas.read_file('https://opendata.arcgis.com/datasets/1f75636917604299861fb408bbf79378_1.geojson')

# Convert it to a coordinate reference system - 3857 is Spheric Mercator
gdf_councils = gdf_councils.to_crs(epsg=3857)

In [3]:
# Download the Champaign tree data, if it doesn't exist in the directory, since this takes a while
tree_file = 'tree_data.geojson'
tree_file_url = 'https://opendata.arcgis.com/datasets/979bbeefffea408e8f1cb7a397196c64_22.geojson'

if os.path.exists(tree_file) and os.stat(tree_file).st_size > 0:
    pass
else:
    data = requests.get(tree_file_url).json()
    with open(tree_file, 'w') as output_file:
        json.dump(data, output_file)
    
gdf_trees = geopandas.read_file(tree_file)

# Keep only the columns we care about, to free up memory and speed processing
useful_cols = ['TREETYPE', 'Special_Status', 'FAMILY', 'INSPECT_DT', 'SPP', 'COMMON', 'geometry']
for c in gdf_trees.columns:
    if c not in useful_cols:
        gdf_trees = gdf_trees.drop(c, axis=1)
        
# Keep only the columns with TREETYPE=Tree (we don't want vacant sites, retired, etc.)
gdf_trees = gdf_trees[gdf_trees['TREETYPE'] == 'Tree']
gdf_trees = gdf_trees.drop('TREETYPE', axis=1)

In [4]:
# Filter down to just oaks, maples, hackberries and ginkgo
# Maples and oaks because they're common and have lots of species, and 
#   hackberries, gingko, sweetgum, and redbud because I like them
gdf_trees = gdf_trees.query('COMMON.str.contains("maple|oak|hackberry|gink|sweetgum|redbud")', engine="python")

# Convert it to a coordinate reference system
gdf_trees = gdf_trees.to_crs(epsg=3857)

# Update the status column
gdf_trees['Special_Status'] = gdf_trees['Special_Status'].fillna('No status')

In [5]:
@ipywidgets.interact(tree_type=['maple', 'oak', 'other'])
def my_geopandas_plot(tree_type):

    if tree_type == 'maple' or tree_type == 'oak':
        gdf_trees_subset = gdf_trees.query('COMMON.str.contains("{}")'.format(tree_type), engine="python")
    else:
        gdf_trees_subset = gdf_trees.query('COMMON.str.contains("hackberry|gink|sweetgum|redbud")', engine="python")


    # Create the grid of plots
    gs_kw = dict(width_ratios=[4, 2], height_ratios=[1, 1])
    fig, axes = plt.subplot_mosaic([['map', 'status'],
                                    ['map', 'inspection']],
                                   gridspec_kw=gs_kw,
                                   figsize=(20, 14), constrained_layout=False)

    # Create the map
    gdf_councils.plot(ax=axes['map'], alpha=0.4, edgecolor='k')
    ctx.add_basemap(ax=axes['map'])
    gdf_trees_subset.plot(ax=axes['map'], column='COMMON', cmap='rainbow', legend=True, categorical=True,)

    # Create the status types
    status_df = gdf_trees_subset['Special_Status'].value_counts().to_frame()
    labels = status_df.index.values.tolist()
    values = status_df['Special_Status'].values.tolist()

    axes['status'].pie(values, labels=labels)
    axes['status'].set_title('Tree Status')

    # Create the inspections
    inspections_df = gdf_trees_subset['INSPECT_DT'].value_counts().to_frame()
    inspections_df.index = inspections_df.index.set_names(['Date'])
    inspections_df = inspections_df.reset_index()
    inspections_df['Date'] = pd.to_datetime(inspections_df['Date'])

    axes['inspection'].plot_date(inspections_df['Date'], inspections_df['INSPECT_DT'])
    axes['inspection'].set_title('Inspection Dates')

    plt.show()

interactive(children=(Dropdown(description='tree_type', options=('maple', 'oak', 'other'), value='maple'), Out…

### Writeup

The tree data for Champaign is quite interesting to interact with.  If I were going to extend the project, I would like to figure out how to turn the trees on and off by district, and also be able to do some data analysis based on whether the tree points were within the district polygons.  That would have taken a lot more familiarity with the map plotting, though.

I tried to put the matplot figure into a ipywidgets box and then realized that I had only done that with bqplot, not maplot.  (Maybe we did it in class, but I was running out of time to double-check.)  I tried using an Output widget, as suggested by https://kapernikov.com/ipywidgets-with-matplotlib/, but it was improperly creating new graphs each time the dropdown menu value changed.  For instance, it would graph for maples, and then if I selected oak, it would put more figures below for oak.  So I used matplotlib subplots instead.